## INSIDE AIRBNB EDA

#### https://insideairbnb.com/

Vamos a abordar este proyecto en dos fases:

- Combinar los 5 archivos CSV en un único dataset con las 4 columnas seleccionadas.

- Realizar un EDA exhaustivo paso a paso, interpretando resultados y tomando decisiones.

#### **1. Objetivo: Unir los archivos en un solo DataFrame con las columnas:**

calendar_last_scraped, room_type, price, neighbourhood_cleansed.

**Comprobamos ubicación de los csv's**

In [ ]:
import os

files = [
    "../data/raw/inside/listings-03-2024.csv",
    "../data/raw/inside/listings-06-2024.csv",
    "data/raw/inside/listings-12-2024.csv" # <- Cambia la ruta a la correcta
]

for file in files:
    if os.path.exists(file):
        print(f"✅ Archivo encontrado: {file}")
    else:
        print(f"❌ Archivo NO encontrado: {file}")

✅ Archivo encontrado: ../data/raw/inside/listings-03-2024.csv
✅ Archivo encontrado: ../data/raw/inside/listings-06-2024.csv
❌ Archivo NO encontrado: data/raw/inside/listings-12-2024.csv


Para realizar un **EDA básico orientado a predecir precios futuros** de alojamientos y evaluar su rentabilidad para inversores, seleccionaré las columnas más relevantes y mínimas posibles, priorizando:

1. **Variables clave para el modelo de precios** (características del inmueble).  
2. **Identificadores únicos** (para evitar duplicados).  
3. **Datos temporales** (para análisis de tendencias).  
4. **Variables de ubicación** (critical en bienes raíces).  

---

### 🔍 **Columnas seleccionadas (8 esenciales)**:  

| **Columna**                  | **Tipo**       | **Razón**                                                                 |
|------------------------------|----------------|---------------------------------------------------------------------------|
| `id`                         | Identificador  | Clave primaria única por listing (evitar duplicados).                     |
| `scrape_id`                  | Temporal       | Identifica lotes de scraping (útil para detectar duplicados por fecha).   |
| `neighbourhood_cleansed`     | Ubicación      | Barrio normalizado (geocodificado), clave para precios.                   |
| `room_type`                  | Característica | Tipo de alojamiento (impacto directo en precio).                          |
| `accommodates`               | Característica | Capacidad de huéspedes (correlación fuerte con precio).                   |
| `bathrooms`                  | Característica | Número de baños (influye en valor).                                       |
| `price`                      | Target         | Variable a predecir (precio diario).                                      |
| `calendar_last_scraped`      | Temporal       | Fecha de última actualización (para análisis de tendencias temporales).   |

---

### 🎯 **Por qué estas columnas (y no otras)**:  
- **`id` + `scrape_id`**:  
  - Garantizan que no hay duplicados (si un `id` se repite en diferentes `scrape_id`, es un error).  
  - Ejemplo de chequeo:  
    ```python
    print(f"¿IDs únicos?: {df['id'].nunique() == len(df)}")  # Debe ser True
    ```  
- **Variables de inmueble (`room_type`, `accommodates`, `bathrooms`)**:  
  - Son **predictores fuertes** del precio (ej: un apartamento entero vale más que una habitación privada).  
- **`neighbourhood_cleansed`**:  
  - Más confiable que `neighbourhood` (está geocodificado). Los precios varían drásticamente por barrio.  
- **`price`**:  
  - Es la variable objetivo (**target**) para el modelo predictivo.  
- **`calendar_last_scraped`**:  
  - Permite analizar si los precios cambian en ciertos períodos (ej: temporada alta vs. baja).  

---

### ⚠️ **Columnas excluidas (pero importantes en fases posteriores)**:  
- **`latitude`/`longitude`**: Útiles para mapas, pero en un EDA inicial complican el análisis.  
- **`number_of_reviews`**: Relevante para popularidad, pero no directamente para precio.  
- **`host_*`**: El anfitrión afecta menos el precio que las características físicas del inmueble.  

---

In [12]:
import pandas as pd

# Listar archivos manualmente (ejemplo)
files = [
    "../data/raw/inside/listings-03-2024.csv",    
    "../data/raw/inside/listings-06-2024.csv",
    "../data/raw/inside/listings-12-2024.csv",
    "../data/raw/inside/listings-03-2025.csv",
    # Agrega todos los archivos necesarios
]

dfs = []
for file in files:
    try:
        df = pd.read_csv(file, usecols=["id", "scrape_id", "neighbourhood_cleansed", "room_type", 
                                        "accommodates", "bathrooms", "price", "calendar_last_scraped"])
        dfs.append(df)
    except Exception as e:
        print(f"Error al leer {file}: {e}")

if dfs:
    df = pd.concat(dfs, ignore_index=True)
    print(df.head())
    # Verificar    
    print(f"Total de registros: {len(df)}")
else:
    print("No se cargaron DataFrames para concatenar.")

       id       scrape_id neighbourhood_cleansed        room_type  \
0   21853  20240322023323               Cármenes     Private room   
1   24805  20240322023323            Universidad  Entire home/apt   
2  204570  20240322023323                 Cortes  Entire home/apt   
3   30320  20240322023323                    Sol  Entire home/apt   
4  205199  20240322023323            Universidad  Entire home/apt   

   accommodates  bathrooms    price calendar_last_scraped  
0             1        1.0   $31.00            2024-03-22  
1             3        1.0   $92.00            2024-03-22  
2             5        2.0  $180.00            2024-03-22  
3             2        NaN      NaN            2024-03-22  
4             2        1.0   $65.00            2024-03-22  
Total de registros: 104996


Pasos críticos:

- Verificar que las columnas existen en todos los archivos.

- Chequear **duplicados** después de la unión (df.duplicated().sum()).

#### **2. EDA**

Vamos a analizar las columnas una por una, detectar problemas, y decidir acciones.

**A. Análisis Inicial**

In [5]:
# Estructura básica
print(df.info())  # Tipos de datos y nulos
print(df.describe(include="all"))  # Estadísticas descriptivas

# Convertir 'price' a numérico (puede tener símbolos como '$' o comas)
#df["price"] = df["price"].replace('[\$,]', '', regex=True).astype(float)
df["price"] = df["price"].replace('[\\$,]', '', regex=True).astype(float)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104996 entries, 0 to 104995
Data columns (total 4 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   neighbourhood_cleansed  104996 non-null  object 
 1   room_type               104996 non-null  object 
 2   price                   82397 non-null   float64
 3   calendar_last_scraped   104996 non-null  object 
dtypes: float64(1), object(3)
memory usage: 3.2+ MB
None
       neighbourhood_cleansed        room_type         price  \
count                  104996           104996  82397.000000   
unique                    128                4           NaN   
top               Embajadores  Entire home/apt           NaN   
freq                    11875            68640           NaN   
mean                      NaN              NaN    136.956879   
std                       NaN              NaN    369.757066   
min                       NaN              NaN      8.000000 

**A.1. Frecuencia relativa**

In [4]:
for col in ['neighbourhood_cleansed', 'room_type', 'price', 'calendar_last_scraped']:
    freq_rel = (df[col].value_counts(normalize=True).iloc[0] * 100)
    top_val = df[col].mode()[0]
    print(f"{col}: '{top_val}' → {freq_rel:.2f}%")

neighbourhood_cleansed: 'Embajadores' → 11.31%
room_type: 'Entire home/apt' → 65.37%
price: '80.0' → 1.26%
calendar_last_scraped: '2024-06-15' → 25.64%


**A.2. Duplicados**

Verificar duplicados exactos (filas idénticas)

In [6]:
# Contar filas duplicadas (exactamente iguales en todas las columnas)
duplicados_totales = df.duplicated().sum()
print(f"Número de filas duplicadas (exactas): {duplicados_totales}")

Número de filas duplicadas (exactas): 58064


In [7]:
# Ejemplo: Verificar duplicados en columnas específicas
columnas_clave = ["neighbourhood_cleansed", "calendar_last_scraped", "room_type"]
duplicados_clave = df.duplicated(subset=columnas_clave).sum()
print(f"Número de duplicados en columnas clave: {duplicados_clave}")

Número de duplicados en columnas clave: 102441


**Volcado de dataframe a csv para comprobar duplicidad de fechas**

In [9]:
df.to_csv("../data/processed/merged-listings.csv", index=False)

In [10]:
# Contar filas EXACTAMENTE iguales en todas las columnas
duplicados_totales = df.duplicated().sum()
print(f"Filas duplicadas (idénticas en TODO): {duplicados_totales}")

Filas duplicadas (idénticas en TODO): 58064
